In [1]:
!pip install requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.3-cp311-cp311-macosx_10_9_universal2.whl.metadata (36 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.3-cp311-cp311-macosx_10_9_universal2.whl (204 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [requests]1/4 [charset_normalizer]


In [9]:
from time import perf_counter

In [15]:
# --------------------------------------------------------------------------
# Comparación Práctica: GraphQL vs REST API en Python (Versión Corregida)
# --------------------------------------------------------------------------
#
# Este script realiza la misma tarea usando dos APIs públicas y funcionales:
# 1. GraphQL: Pide exactamente los datos que necesita a la API de Países.
# 2. REST: Obtiene todos los datos de la API REST de Países y los filtra.
#
# Para ejecutar este ejemplo, solo necesitas la librería 'requests':
# pip install requests
#
# Luego, guarda este código y ejecútalo, por ejemplo:
# python compare_apis.py
# --------------------------------------------------------------------------

import requests
import json

# --- Parte 1: Enfoque con GraphQL ---

def fetch_data_with_graphql():
    """
    Obtiene datos de países usando una consulta GraphQL.
    """
    print("Realizando petición con GraphQL a la API de Países...")
    print("-" * 60)

    # URL de una API de GraphQL pública y estable sobre países.
    api_url = "https://countries.trevorblades.com/"

    # La consulta pide EXCLUSIVAMENTE los 3 campos que nos interesan para 5 países.
    graphql_query = """
    query ObtenerPaises {
      countries(filter: { code: { in: ["ES", "MX", "AR", "CO", "PE"] } }) {
        name
        capital
        currency
      }
    }
    """

    try:
        response = requests.post(api_url, json={'query': graphql_query})
        response.raise_for_status()
        data = response.json()
        size_gql = len(response.content)
        print(f"Cantidad de países obtenidos: {len(data['data']['countries'])}")
        print(f"Tamaño de la respuesta GraphQL: {size_gql} bytes")

        if 'data' in data and data['data']['countries']:
            paises = data['data']['countries']
            print("Datos obtenidos con GraphQL:")
            for pais in paises:
                print(f"  - País: {pais.get('name', 'N/A')}, "
                      f"Capital: {pais.get('capital', 'N/A')}, "
                      f"Moneda: {pais.get('currency', 'N/A')}")
        elif 'errors' in data:
            print(f"Error en la respuesta de GraphQL: {data['errors']}")

    except requests.exceptions.RequestException as err:
        print(f"Ocurrió un error en la petición GraphQL: {err}")

# --- Parte 2: Enfoque con REST API ---

def fetch_data_with_rest():
    """
    Obtiene datos de países usando una API REST tradicional.
    """
    print("\nRealizando petición con REST a la API de Países...")
    print("-" * 60)

    # URL de una API REST pública y muy completa sobre países.
    api_url = "https://restcountries.com/v3.1/all?fields=name,capital,currencies"

    try:
        response = requests.get(api_url)
        response.raise_for_status()
        paises_completos = response.json()
        size_rest = len(response.content)
        print(f"Total de países obtenidos: {len(paises_completos)}")
        print(f"Tamaño de la respuesta REST: {size_rest} bytes")

        # En este caso, la API REST nos permite filtrar campos con ?fields=...
        # Sin embargo, la estructura de la respuesta sigue siendo más compleja
        # que la de GraphQL y a menudo contiene más datos de los necesarios.

        print("Datos obtenidos con REST. Ahora los procesamos:")
        # Filtramos para mostrar los mismos 5 países que en el ejemplo de GraphQL
        codigos_deseados = ["Spain", "Mexico", "Argentina", "Colombia", "Peru"]
        for pais in paises_completos:
            if pais.get("name", {}).get("common") in codigos_deseados:
                # La estructura de la respuesta REST es más anidada.
                nombre = pais.get("name", {}).get("common", "N/A")
                # La capital es una lista, tomamos el primer elemento.
                capital = pais.get("capital", ["N/A"])[0]
                # Las monedas son un diccionario, necesitamos extraer el nombre.
                monedas = pais.get("currencies", {})
                # Obtenemos la primera clave del diccionario de monedas (ej. 'EUR')
                codigo_moneda = next(iter(monedas.keys()), None)
                nombre_moneda = monedas.get(codigo_moneda, {}).get("name", "N/A") if codigo_moneda else "N/A"

                print(f"  - País: {nombre}, "
                      f"Capital: {capital}, "
                      f"Moneda: {nombre_moneda}")
                

    except requests.exceptions.RequestException as err:
        print(f"Ocurrió un error en la petición REST: {err}")

def measure_time(func):
    """
    Mide el tiempo de ejecución de una función.
    """
    start_time = perf_counter()
    func()
    end_time = perf_counter()
    elapsed_time = end_time - start_time
    print(f"Tiempo de ejecución: {elapsed_time:.4f} segundos\n")
    return elapsed_time

# --- Ejecución Principal ---
if __name__ == "__main__":
    time_gql = measure_time(fetch_data_with_graphql)
    time_rest = measure_time(fetch_data_with_rest)

    print("\n--- ANÁLISIS DE LA COMPARACIÓN ---")
    print("1. GraphQL: La consulta es precisa. Recibimos una respuesta limpia y plana con exactamente lo que pedimos.")
    print("2. REST: Aunque filtramos campos en la URL, la respuesta tiene una estructura más compleja y anidada que requiere más procesamiento en nuestro código para extraer los datos.")
    print("3. Flexibilidad: Con GraphQL, si queremos datos relacionados (ej. idiomas), simplemente los añadimos a la consulta. Con REST, dependemos de si el endpoint nos proporciona esa información o si tenemos que hacer otra llamada a un endpoint diferente.")

    print("\n--- COMPARACIÓN TEMPORAL ---")
    print(f"Tiempo total con GraphQL: {time_gql:.4f} segundos")
    print(f"Tiempo total con REST: {time_rest:.4f} segundos")


Realizando petición con GraphQL a la API de Países...
------------------------------------------------------------
Cantidad de países obtenidos: 5
Tamaño de la respuesta GraphQL: 306 bytes
Datos obtenidos con GraphQL:
  - País: Argentina, Capital: Buenos Aires, Moneda: ARS
  - País: Colombia, Capital: Bogotá, Moneda: COP
  - País: Spain, Capital: Madrid, Moneda: EUR
  - País: Mexico, Capital: Mexico City, Moneda: MXN
  - País: Peru, Capital: Lima, Moneda: PEN
Tiempo de ejecución: 0.3902 segundos


Realizando petición con REST a la API de Países...
------------------------------------------------------------
Total de países obtenidos: 250
Tamaño de la respuesta REST: 71942 bytes
Datos obtenidos con REST. Ahora los procesamos:
  - País: Colombia, Capital: Bogotá, Moneda: Colombian peso
  - País: Peru, Capital: Lima, Moneda: Peruvian sol
  - País: Spain, Capital: Madrid, Moneda: Euro
  - País: Argentina, Capital: Buenos Aires, Moneda: Argentine peso
  - País: Mexico, Capital: Mexico City,

https://github.com/graphql-kit/graphql-apis
